<a href="https://colab.research.google.com/github/YuxuanHuang455/Micron-BERT/blob/main/week2_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新段落

In [1]:
import torch, platform, sys
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
CUDA available: False


In [2]:
!pip -q install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
!pip -q install pandas numpy scikit-learn opencv-python matplotlib tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os, pandas as pd
DATA_ROOT = "/content/data"
os.makedirs(DATA_ROOT, exist_ok=True)

# 下载 fer2013.csv
!wget -q -O {DATA_ROOT}/fer2013.csv https://raw.githubusercontent.com/muxspace/facial_expressions/master/fer2013/fer2013.csv
assert os.path.exists(f"{DATA_ROOT}/fer2013.csv")
df = pd.read_csv(f"{DATA_ROOT}/fer2013.csv")
df.head(), df["Usage"].value_counts(), df["emotion"].value_counts()


EmptyDataError: No columns to parse from file

In [ ]:
import torch, numpy as np, cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class FER2013Dataset(Dataset):
    def __init__(self, csv_path, split="Training", img_size=224, augment=False):
        df = pd.read_csv(csv_path)
        df = df[df["Usage"]==split]
        self.pixels = df["pixels"].tolist()
        self.labels = df["emotion"].values.astype(np.int64)
        t = []
        if augment and split=="Training":
            t += [transforms.RandomHorizontalFlip(p=0.5)]
        t += [
            transforms.ToPILImage(),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            # ImageNet 归一化，方便迁移学习
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ]
        self.tf = transforms.Compose(t)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        pix = np.fromstring(self.pixels[idx], dtype=np.uint8, sep=' ').reshape(48,48)
        # 灰度 → 3 通道
        img = cv2.cvtColor(pix, cv2.COLOR_GRAY2RGB)
        x = self.tf(img)
        y = self.labels[idx]
        return x, y

img_size = 224
train_ds = FER2013Dataset(f"{DATA_ROOT}/fer2013.csv", split="Training", img_size=img_size, augment=True)
val_ds   = FER2013Dataset(f"{DATA_ROOT}/fer2013.csv", split="PublicTest", img_size=img_size, augment=False)
test_ds  = FER2013Dataset(f"{DATA_ROOT}/fer2013.csv", split="PrivateTest", img_size=img_size, augment=False)

bs = 128
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=bs, shuffle=False, num_workers=2, pin_memory=True)

len(train_ds), len(val_ds), len(test_ds)


In [ ]:
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

device = "cuda" if torch.cuda.is_available() else "cpu"
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 7)
model = model.to(device)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [ ]:
from tqdm import tqdm
import numpy as np

def run_epoch(loader, train=True):
    model.train(train)
    total, correct, loss_sum = 0, 0, 0.0
    for x,y in tqdm(loader, disable=False):
        x,y = x.to(device), y.to(device)
        if train:
            optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = criterion(logits, y)
        if train:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        loss_sum += loss.item()*y.size(0)
        pred = logits.argmax(1)
        total += y.size(0)
        correct += (pred==y).sum().item()
    return loss_sum/total, correct/total

best_val = 0
EPOCHS = 3  # 为了快，先跑 3 个 epoch；有时间可以增大
for ep in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc = run_epoch(val_loader, train=False)
    print(f"Epoch {ep}: train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {val_loss:.4f} acc {val_acc:.4f}")
    if val_acc > best_val:
        best_val = val_acc
        torch.save(model.state_dict(), "/content/best_fer2013_resnet18.pt")
print("Best val acc:", best_val)


In [ ]:
# 载入最佳权重并在测试集评估
state = torch.load("/content/best_fer2013_resnet18.pt", map_location=device)
model.load_state_dict(state)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for x,y in tqdm(test_loader):
        x = x.to(device)
        logits = model(x)
        pred = logits.argmax(1).cpu().numpy().tolist()
        y_pred += pred
        y_true += y.numpy().tolist()

print("Test Acc:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=4))
cm = confusion_matrix(y_true, y_pred)
cm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/best_fer2013_resnet18.pt /content/drive/MyDrive/best_fer2013_resnet18.pt
print("Saved to Drive: /content/drive/MyDrive/best_fer2013_resnet18.pt")
